## Create Network Layers

The GNN consists of two layers. The first layer embeds the attributes in various nodes. The second layer is a latent (hidden) representation of the nodes. No dropout is used currently. We should explore this

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn

class HeteroRGCNLayer1(nn.Module):
    EMBED_SIZE = 64
    VOCAB_SIZE = 2386
    def __init__(self, hidden_size, G):
        super(HeteroRGCNLayer1, self).__init__()
        # Need an embedding layer for each node feature
        self.node_embeddings = {}
        #self.dropouts = {}
        for ntype in G.ntypes:
            # create an embedding for each feature of a node
            self.node_embeddings[ntype] = {}
            num_node_features = G.node_attr_schemes(ntype)['f'].shape[0]
            for feature in range(num_node_features):
                self.node_embeddings[ntype][feature] = nn.Embedding(self.VOCAB_SIZE, self.EMBED_SIZE)
            #self.dropouts[ntype] = nn.Dropout()
        #for name in etypes:
        module_layers = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            num_features = G.node_attr_schemes(srctype)['f'].shape[0]
            module_layers[etype] = nn.Linear(num_features * self.EMBED_SIZE, hidden_size)
        self.weight = nn.ModuleDict(module_layers)
        self.hidden_size = hidden_size

    def forward(self, G):

        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # for each node compute the embedding and store it in the graph
            # iterate over the features of each node and compute the embedding
            the_node_embedding = self.node_embeddings[srctype]
            node_feature_embeddings = []
            num_features = G.node_attr_schemes(srctype)['f'].shape[0]
            for feature in range(num_features):
                feature_embedding_layer = the_node_embedding[feature]
                node_feature_embeddings.append(feature_embedding_layer(G.nodes[srctype].data['f'][:, feature]))
            comp_node_embedding = torch.cat(node_feature_embeddings, 1)
            G.nodes[srctype].data['E'] = comp_node_embedding
            # Compute W_r * h
            Wh = self.weight[etype](G.nodes[srctype].data['E'])  
            #Wh = torch.sum(Wh, dim = 1)
                # Save it in graph for message passing
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        #return G
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}

class HeteroRGCNLayer2(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer2, self).__init__()
        # W_r for each relation
        
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h
            Wh = self.weight[etype](feat_dict[srctype])
            # Save it in graph for message passing
            G.nodes[srctype].data['Wh2_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh2_%s' % etype, 'm'), fn.mean('m', 'h2'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        #return G
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h2'] for ntype in G.ntypes}

## Load the graph
The graph is loaded using the ITSM_Dataloader. This class:
1. Creates the graph schema for the ITSM data in Arango DB
2. Loads the data into Arango DB
3. Creates a Graph in the format required by the DGL graph library

In [ ]:
# generate test and train indices
from ITSM_data_loader import ITSM_Dataloader

dl = ITSM_Dataloader(create_db = False)
labels, G = dl.get_networkx_graph(load_from_db=True, graph_desc_file_loc= "graph_descriptor.yaml")

In [ ]:
G

## Verify Graph Properties

In [ ]:
G.canonical_etypes

In [ ]:
# Draw the metagraph using graphviz.
import networkx as nx
import matplotlib.pyplot as plt 
%matplotlib inline
nx.draw_networkx(G.metagraph)

In [ ]:
G.node_attr_schemes('incident')

## Define Heterogeneous Graph Convolution Neural Network (HGCN)

In [ ]:
class HeteroRGCN(nn.Module):
    def __init__(self, G, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # create layers
        self.layer1 = HeteroRGCNLayer1(hidden_size, G)
        self.layer2 = HeteroRGCNLayer2(hidden_size, out_size, G.etypes)

        

    def forward(self, G):
        
        h_dict = self.layer1(G)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        
        # get paper logits
        
        return h_dict['incident']

## Create test and train data

In [ ]:
training_mask = np.random.rand(len(labels)) <= 0.8
train_idx = [i for i in range(len(labels)) if training_mask[i]]
test_idx = [i for i in range(len(labels)) if not training_mask[i]]
train_idx = torch.tensor(train_idx).long()
test_idx = torch.tensor(test_idx).long()
labels = torch.tensor(labels).long()

## Train the HGCN and make predictions

In [ ]:
%%time
# Create the model. The output has three logits for three classes.
model = HeteroRGCN(G,32,2)
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()

for epoch in range(100):
    opt.zero_grad()
    logits = model(G)
    # The loss is computed only for labeled nodes.
    loss = loss_fn(logits[train_idx], labels[train_idx])
    loss.backward()
    opt.step()
    pred_trng = torch.argmax(logits[train_idx], dim = 1)
    res_trng = pred_trng == labels[train_idx]
    trng_acc = torch.sum(res_trng).item()/labels[train_idx].shape[0]
    
    pred_test = torch.argmax(logits[test_idx], dim = 1)
    res_test = pred_test == labels[test_idx]
    test_acc = torch.sum(res_test).item()/labels[test_idx].shape[0]
    
    
   
    if epoch % 10 == 0:
        print( 'Loss %.4f, training accuracy %.4f, test accuracy %.4f' % ( loss.item(), trng_acc, test_acc ) )